In [6]:
#Lettura del file assets_dataset.csv
File_Assets=open("assets_dataset.csv", "r")
File_Cov=open("./assets_cov_matrix.csv", "r")
vet = {}
for count, i in enumerate(File_Assets):
    if count!=0:
        id=i.split("_")[1].split(",")[0]
        num1=i.split(",")[1].strip()
        num2=i.split(",")[2].strip()
        vet[int(id)]=[float(num1),float(num2)]

In [7]:
#Lettura del file assets_cov_matrix.csv
MatCov=[]
n=[]
for count, i in enumerate(File_Cov):
    if count!=0:
        n=i.split(",")
        for j in range(500-1):
            n[j]=float(n[j+1])
        MatCov.append(n)

In [13]:
import qubovert as QbVrt
from qubovert.sim import anneal_qubo

n_assets = 500
precision = 12

#Creazione delle variabili booleane
#x_i=1 se l'asset i è presente nel portafoglio, x_i=0 altrimenti
def create_boolean_variables(n_assets):
    # Creazione della matrice delle variabili booleane, inizializzate a None
    var_matrix = vars = [[None for j in range(n_assets)] for i in range(n_assets)]
    for i in range(n_assets):
        for j in range (n_assets):
            var = QbVrt.boolean_var(f"x_{i}{j}")	
            var_matrix[i][j]=var;
    return var_matrix

model_H0 = QbVrt.QUBO()
model_H1 = QbVrt.QUBO()

coeff = []

# creazione della funzione obiettivo per H0
def create_H0(n_assets, precision, MatCov, model_H0, coeff):
    for i in range(n_assets):
        for j in range(i, n_assets):
            for k in range(precision-1):
                for l in range(precision-1):
                    model_H0-=MatCov[i][j]*(coeff[k]*x[i][k])*(coeff[l]*x[j][l])
    return model_H0
    


#solution = anneal_qubo(model,  num_anneals=10)




0,0,0,1,0

IndexError: list index out of range

In [14]:

print(create_boolean_variables(5))

[[None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None]]
[[{('x_00',): 1}, {('x_01',): 1}, {('x_02',): 1}, {('x_03',): 1}, {('x_04',): 1}], [{('x_10',): 1}, {('x_11',): 1}, {('x_12',): 1}, {('x_13',): 1}, {('x_14',): 1}], [{('x_20',): 1}, {('x_21',): 1}, {('x_22',): 1}, {('x_23',): 1}, {('x_24',): 1}], [{('x_30',): 1}, {('x_31',): 1}, {('x_32',): 1}, {('x_33',): 1}, {('x_34',): 1}], [{('x_40',): 1}, {('x_41',): 1}, {('x_42',): 1}, {('x_43',): 1}, {('x_44',): 1}]]
